In [2]:
# Account login IDs (not credentials) imported from a variables file

. ./AZ_Sub_Details_1.ps1

$tenantID = $TENANTID_DEF
$subscriptionID = $SUBSCRIPTIONID_DEF
$location="EastUS2"

## Network information
$vNetName="prd-vnte2dem001"
$vNetAddPref="192.168.0.0/16"
$sbt1Name="iaassbt"
$sbt1AddPref="192.168.1.0/24"

#if NEEDED 
#$VMLocalAdminUser = ""
#$VMLocalAdminSecurePassword = ConvertTo-SecureString  -AsPlainText -Force
$resourceGroupName="rg-prdsecdemo001"
$newVMName="cpt-prde2dem001"
$VMSize = "Standard_B2s"
$newVMPubName = "Canonical"
$newVMOffer = "UbuntuServer"
$newVMSku = "18.04-LTS"
$NICName = $newVMName+"nic1"

$vmDiskSize = 5
$vmDiskSkuName = "Standard_LRS"
$vmDiskNm = "OSDisk1"

In [3]:
## Connect to Azure
Connect-AzAccount -Tenant $tenantID -SubscriptionId $subscriptionID


Account               SubscriptionName         TenantId                             Environment
-------               ----------------         --------                             -----------
matperr@microsoft.com Visual Studio Enterprise cc914548-7923-4586-ad62-f99cebba3dac AzureCloud



In [ ]:
## If Resource Group Exists do not create, else create
New-AzResourceGroup -Name $resourceGroupName -Location $location

In [ ]:
## Create Network resources
## Source: https://docs.microsoft.com/en-us/powershell/module/az.network/new-azvirtualnetwork?view=azps-5.4.0
$sbt1  = New-AzVirtualNetworkSubnetConfig -Name $sbt1Name  -AddressPrefix $sbt1AddPref
$vNet = New-AzVirtualNetwork -Name $vNetName -ResourceGroupName $resourceGroupName -Location $location -AddressPrefix $vNetAddPref -Subnet $sbt1

In [ ]:
## Create VM - note this configuration is demo dependant like building a LINUX OS
## Source: https://docs.microsoft.com/en-us/powershell/module/az.compute/new-azvm?view=azps-5.4.0

$Credential = Get-Credential
## $Credential = New-Object System.Management.Automation.PSCredential ($VMLocalAdminUser, $VMLocalAdminSecurePassword);

$NIC = New-AzNetworkInterface -Name $NICName -ResourceGroupName $ResourceGroupName -Location $Location -SubnetId $vNet.Subnets[0].id

$VirtualMachine = New-AzVMConfig -VMName $newVMName -VMSize $VMSize
#$VirtualMachine = Set-AzVMOperatingSystem -VM $VirtualMachine -Windows -ComputerName $newVMName -Credential $Credential -ProvisionVMAgent -EnableAutoUpdate
$VirtualMachine = Set-AzVMOperatingSystem -VM $VirtualMachine -Linux -ComputerName $newVMName -Credential $Credential
$VirtualMachine = Add-AzVMNetworkInterface -VM $VirtualMachine -Id $NIC.Id
$VirtualMachine = Set-AzVMSourceImage -VM $VirtualMachine -PublisherName $newVMPubName -Offer $newVMOffer -Skus $newVMSku -Version latest
##$VirtualMachine = Set-AzVMOSDisk -VM $VirtualMachine -Name $vmDiskNm -Linux -CreateOption "Attach"

New-AzVM -ResourceGroupName $ResourceGroupName -Location $Location -VM $VirtualMachine -Verbose

